In [2]:
def zeroinfl(endog, exog_count, exog_zero, dist = 'poisson', link = 'logit', weights = None, offsetx = None,\
             offsetz = None, method = 'L-BFGS-B', start = None, EM = True, \
                                        tol = None, options = None, factr = 1.0):
    ## Function starts
    import numpy as np
    import pandas as pd
    import statsmodels.api as sm
    import scipy as sp
    import scipy.stats as st
    import sys
    import warnings

    FLOAT_EPS = np.finfo(float).eps
    
    Y = endog
    X = exog_count
    Z = exog_zero
    ## convenience variables
    Y = np.squeeze(y.values)
    n = len(Y)
    kx = X.shape[1] # Number of columns in X
    kz = Z.shape[1]
    Y0 = Y <= 0
    Y1 = Y > 0
    
    ## sanity checks
    if len(Y) < 1:
        sys.exit("empty model")
    if np.all(Y > 0):
        sys.exit("invalid dependent variable, minimum count is not zero")  
    if np.array_equal(np.asarray(Y), (np.round(Y + 0.001)).astype(int)) is False:
        sys.exit("invalid dependent variable, non-integer values")
    Y = (np.round(Y + 0.001)).astype(int)
    if np.any(Y < 0):
        sys.exit("invalid dependent variable, negative counts")

    ## weights and offset

    if weights is None:
        weights = 1.0
    weights = np.ndarray.flatten(np.array(weights))
    if weights.size == 1:
        weights = np.repeat(weights,n)
    weights = pd.Series(data = weights, index = X.index)

    if offsetx is None:
        offsetx = 0.0
    offsetx = np.ndarray.flatten(np.array(offsetx))
    if offsetx.size == 1:
        offsetx = np.repeat(offsetx,n)

    if offsetz is None:
        offsetz = 0.0
    offsetz = np.ndarray.flatten(np.array(offsetz))
    if offsetz.size == 1:
        offsetz = np.repeat(offsetz,n)
        
    class Logit(object):
        def __init__(self):
            self.linkclass = sm.genmod.families.links.logit
        def link(self, mu):
            return mu/(1.0 + mu)
        def link_inv(self, eta):
            thresh = 30.0
            eta = np.minimum(np.maximum(eta,-thresh), thresh)
            exp_eta = np.exp(eta)
            return exp_eta/(1+exp_eta)
        def link_inv_deriv(self, eta):
            thresh = 30.0
            eta[abs(eta) > thresh] = FLOAT_EPS
            return np.exp(eta)/(1+np.exp(eta))**2

    class Probit(object):
        def __init__(self):
            self.linkclass = sm.genmod.families.links.probit
        def link(self, mu):
            return st.norm.ppf(mu)
        def link_inv(self, eta):
            thresh = -st.norm.ppf(FLOAT_EPS)
            eta = np.minimum(np.maximum(eta,-thresh),thresh)
            return st.norm.cdf(eta)
        def link_inv_deriv(self, eta):
            return np.maximum(st.norm.pdf(eta),FLOAT_EPS)
    
    class CLogLog(object):
        def __init__(self):
            self.linkclass = sm.genmod.families.links.cloglog
        def link(self, mu):
            return np.log(-np.log(1 - mu))
        def link_inv(self, eta):
            return np.maximum(np.minimum(-np.expm1(-np.exp(eta)),1-FLOAT_EPS),FLOAT_EPS)
        def link_inv_deriv(self, eta):
            eta = np.minimum(eta,700)
            return np.maximum(np.exp(eta)*np.exp(-np.exp(eta)),FLOAT_EPS)
    
    class Cauchit(object):
        def __init__(self):
            self.linkclass = sm.genmod.families.links.cauchy
        def link(self, mu):
            return st.cauchy.ppf(mu)
        def link_inv(self, eta):
            thresh = -st.cauchy.ppf(FLOAT_EPS)
            eta = np.minimum(np.maximum(eta,-thresh),thresh)
            return st.cauchy.cdf(eta)
        def link_inv_deriv(self, eta):
            return nnp.maximum(st.cauchy.pdf(eta),FLOAT_EPS)
    
    class Log(object):
        def __init__(self):
            self.linkclass = sm.genmod.families.links.log
        def link(self, mu):
            return np.log(mu)
        def link_inv(self, eta):
            return np.maximum(np.exp(eta), FLOAT_EPS)
        def link_inv_deriv(self, eta):
            return np.maximum(np.exp(eta), FLOAT_EPS)
        
    def setLinkClass(argument):
        Link = {
            'logit': Logit(),
            'probit': Probit(),
            'cloglog': CLogLog(),
            'cauchit': Cauchit(),
            'log': Log(),
        }
        return Link.get(argument, Logit())
    
    ## binary link processing
    linkstr = link
    linkobj = setLinkClass(linkstr)
    linkList = ['logit','probit','cauchit','cloglog','log']
    if linkstr not in linkList:
        warnings.warn(linkstr +" link not valid. Available links are: " + str(linkList))
        linkstr = 'logit'

    def ziPoisson(parms, sign = 1.0):
        """
        Log-likelihood of Zero Inflated Poisson.
        """
        ## count mean
        mu = np.exp(np.dot(X,parms[np.arange(kx)]) + offsetx)
        ## binary mean
        phi = linkobj.link_inv(np.dot(Z, parms[np.arange(kx,kx+kz)]) + offsetz)
        ## log-likelihood for y = 0 and y >= 1
        loglik0 = np.log(phi + np.exp(np.log(1-phi) - mu)) 
        loglik1 = np.log(1-phi) + sp.stats.poisson.logpmf(Y, mu)
        ## collect and return
        loglik = np.dot(weights[Y0],loglik0[Y0])+np.dot(weights[Y1],loglik1[Y1])
        return sign*loglik

    def gradPoisson(parms, sign = 1.0):
        """
        Gradient of Zero Inflated Poisson Log-likelihood.
        """
        ## count mean
        eta = np.dot(X,parms[np.arange(kx)]) + offsetx
        mu = np.exp(eta)
        ## binary mean
        etaz = np.dot(Z, parms[np.arange(kx,kx+kz)]) + offsetz
        muz = linkobj.link_inv(etaz)
        ## densities at 0
        clogdens0 = -mu
        dens0 = muz*(1-Y1.astype(float)) + np.exp(np.log(1 - muz) + clogdens0)
        ## working residuals  
        wres_count = np.where(Y1,Y-mu,-np.exp(-np.log(dens0) + 
                                          np.log(1 - muz) + clogdens0 + np.log(mu))) 
        link_etaz = linkobj.link_inv_deriv(etaz)
        wres_zero  = np.where(Y1,-1/(1-muz) * link_etaz, \
                          (link_etaz - np.exp(clogdens0) * link_etaz)/dens0)   
    
        return sign*(np.hstack((np.expand_dims(wres_count*weights,axis=1)*X, \
                np.expand_dims(wres_zero*weights,axis=1)*Z))).sum(axis=0)
    
    def convert_params(mu, theta):
        """
        Convert mean/dispersion parameterization of a negative binomial to the ones scipy supports

        """
        r = theta
        var = mu + 1 / r * mu ** 2
        p = (var - mu) / var
        return r, 1 - p
    
    def ziNegBin(parms, sign = 1.0):
        """
        Log-Likelihood of Zero Inflated Negative Binomial.
        """
        ## count mean
        mu = np.exp(np.dot(X,parms[np.arange(kx)]) + offsetx)
        ## binary mean
        phi = linkobj.link_inv(np.dot(Z, parms[np.arange(kx,kx+kz)]) + offsetz)
        ## negbin size
        theta = np.exp(parms[kx+kz])
    
        ## log-likelihood for y = 0 and y >= 1 sp.stats.poisson.logpmf(Y, mu)
        loglik0 = np.log(phi + np.exp(np.log(1-phi) + \
                                   st.nbinom.logpmf(0,*convert_params(theta = theta, mu = mu)) ) )
        loglik1 = np.log(1-phi) + st.nbinom.logpmf(Y,*convert_params(theta = theta, mu = mu))

        ## collect and return
        loglik = np.dot(weights[Y0],loglik0[Y0])+np.dot(weights[Y1],loglik1[Y1])
        return sign*loglik
  
    def ziGeom(parms, sign = 1.0):
        return ziNegBin(np.hstack((parms, 0)), sign)
    
    def gradGeom(parms, sign = 1.0):
        """
        Gradient of Zero Inflated Geometric Log-likelihood.
        
        """
        ## count mean
        eta = np.dot(X,parms[np.arange(kx)]) + offsetx
        mu = np.exp(eta)
        ## binary mean
        etaz = np.dot(Z, parms[np.arange(kx,kx+kz)]) + offsetz
        muz = linkobj.link_inv(etaz) 

        ## densities at 0
        clogdens0 = st.nbinom.logpmf(0,*convert_params(theta = 1, mu = mu))
        dens0 = muz*(1-Y1.astype(float)) + np.exp(np.log(1 - muz) + clogdens0)

        ## working residuals  
        wres_count = np.where(Y1,Y - mu*(Y + 1)/(mu + 1), \
                              -np.exp(-np.log(dens0) + np.log(1 - muz) + clogdens0 +\
                                      -np.log(mu+1) + np.log(mu))) 
        link_etaz = linkobj.link_inv_deriv(etaz)
        wres_zero  = np.where(Y1,-1/(1-muz) * link_etaz, \
                          (link_etaz - np.exp(clogdens0) * link_etaz)/dens0)
      
        return sign*(np.hstack((np.expand_dims(wres_count*weights,axis=1)*X, \
                np.expand_dims(wres_zero*weights,axis=1)*Z))).sum(axis=0)
    
    def gradNegBin(parms, sign = 1.0): 
        """
        Gradient of Zero Inflated Negative Binomial Log-likelihood. 
        (Negetive Binomial2 to be specific.)
        
        """
        ## count mean
        eta = np.dot(X,parms[np.arange(kx)]) + offsetx
        mu = np.exp(eta)
        ## binary mean
        etaz = np.dot(Z, parms[np.arange(kx,kx+kz)]) + offsetz
        muz = linkobj.link_inv(etaz)    
        ## negbin size
        theta = np.exp(parms[kx+kz])

        ## densities at 0
        clogdens0 = st.nbinom.logpmf(0,*convert_params(theta = theta, mu = mu))
        dens0 = muz*(1-Y1.astype(float)) + np.exp(np.log(1 - muz) + clogdens0)
        
        ## working residuals  
        wres_count = np.where(Y1,Y - mu*(Y + theta)/(mu + theta), \
                              -np.exp(-np.log(dens0) + np.log(1 - muz) + clogdens0 + np.log(theta) +\
                                      -np.log(mu+theta) + np.log(mu))) 
        link_etaz = linkobj.link_inv_deriv(etaz)
        wres_zero  = np.where(Y1,-1/(1-muz) * link_etaz, \
                          (link_etaz - np.exp(clogdens0) * link_etaz)/dens0)
        
        wres_theta = theta*np.where(Y1, sp.special.digamma(Y + theta) - sp.special.digamma(theta) +\
                                   np.log(theta) - np.log(mu + theta) + 1 - (Y + theta)/(mu + theta),\
                                   np.exp(-np.log(dens0) + np.log(1 - muz) + clogdens0)*\
                                   (np.log(theta) - np.log(mu + theta) + 1 - theta/(mu+theta) ) )
        
        return sign*(np.hstack((np.expand_dims(wres_count*weights,axis=1)*X, \
                np.expand_dims(wres_zero*weights,axis=1)*Z, \
                               np.expand_dims(wres_theta,axis=1)))).sum(axis=0)

    
    reltol = tol
    if factr < 1.0:
        warnings.warn('Minimum value of factr is 1.0.')
        factr = 1.0
    if reltol is None:
        reltol = factr*(np.finfo(float).eps)**(1/1.6)

    
    if dist not in ['poisson','negbin','geom']:
        sys.exit(dist+" method not yet implemented")
    if dist is 'poisson':
        loglikfun = ziPoisson
        gradfun = gradPoisson
    elif dist is 'negbin':
        loglikfun = ziNegBin
        gradfun = gradNegBin
    else:
        loglikfun = ziGeom
        gradfun = gradGeom
        
    if options is None:
        options = {'disp': False, 'maxiter': 100000}


    # starting values
    if start is not None:
        valid = True
        if ('count' in start) is False:
            valid = False
            warnings.warn("invalid starting values, count model coefficients not specified")
            start['count'] = pd.Series(np.repeat(0,kx), index = X.columns.values)
        if ('zero' in start) is False:
            valid = False
            warnings.warn("invalid starting values, zero model coefficients not specified")
            start['zero'] = pd.Series(np.repeat(0,kz), index = Z.columns.values)
        if len(start['count']) != kx:
            valid = False
            warnings.warn("invalid starting values, wrong number of count model coefficients")
        if len(start['zero']) != kz:
            valid = False
            warnings.warn("invalid starting values, wrong number of zero model coefficients")
        if dist is 'negbin':
            if ('theta' in start) is False:
                start['theta'] = 1.0
            start = {'zero':start['zero'], 'count':start['count'], 'theta' : (start['theta'][0]).astype(float)}
        else:
            start = {'zero':start['zero'], 'count':start['count']}    
        
        if valid is False:
            start = None

    if start is None:
    ## EM estimation of starting values
        
        model_count = sm.GLM(endog = Y, exog = X, family = sm.families.Poisson(),\
                                  offset = offsetx , freq_weights = weights).fit()
        model_zero = sm.GLM(Y0.astype(int), exog = Z, family=sm.families.Binomial(link = linkobj.linkclass), \
                   offset = offsetz , freq_weights = weights).fit()
        start = {'zero':model_zero.params, 'count':model_count.params}
        
        if dist is 'negbin':
            start['theta'] = 1.0 
            
        if (EM is True) and (dist is 'poisson'):
            mui = model_count.predict()
            probi = model_zero.predict()
            probi = probi/(probi + (1-probi)*sp.stats.poisson.pmf(0, mui))
            probi[Y1] = 0
            probi
            ll_new = loglikfun(np.hstack((start['count'].values,start['zero'].values)))
            ll_old = 2 * ll_new
    
            while np.absolute((ll_old - ll_new)/ll_old) > reltol :
                ll_old = ll_new
                model_count = sm.GLM(endog = Y, exog = X, family = sm.families.Poisson(),\
                                  offset = offsetx , freq_weights = weights*(1-probi) \
                                              ).fit(start_params = start['count'].values)        
                model_zero = sm.GLM(probi, exog = Z, family=sm.families.Binomial(link = linkobj.linkclass),\
                        offset = offsetz, freq_weights = weights \
                               ).fit(start_params = start['zero'].values)
                start = {'zero':model_zero.params, 'count':model_count.params}

                mui = model_count.predict()
                probi = model_zero.predict()
                probi = probi/(probi + (1-probi)*sp.stats.poisson.pmf(0, mui))
                probi[Y1] = 0

                ll_new = loglikfun(np.hstack((start['count'].values,start['zero'].values)))           
            
        if (EM is True) and (dist is 'geom'):
            mui = model_count.predict()
            probi = model_zero.predict()
            probi = probi/(probi + (1-probi)*st.nbinom.pmf(0,*convert_params(theta = 1, mu = mui)))
            probi[Y1] = 0
            
            ll_new = loglikfun(np.hstack((start['count'].values,start['zero'].values)))
            ll_old = 2 * ll_new  
                           
            while np.absolute((ll_old - ll_new)/ll_old) > reltol :
                ll_old = ll_new
                model_count = sm.GLM(endog = Y, exog = X, family = sm.families.NegativeBinomial(alpha = 1.0),\
                                  offset = offsetx , freq_weights = weights*(1-probi)).fit(\
                                        #start_params = start['count'].values
                                    sm.families.NegativeBinomial(alpha = 1.0\
                                                                ).starting_mu(y=start['count'].values))
                model_zero = sm.GLM(probi, exog = Z, family=sm.families.Binomial(link = linkobj.linkclass),\
                        offset = offsetz, freq_weights = weights).fit(start_params = start['zero'].values)
                start = {'zero':model_zero.params, 'count':model_count.params}

                mui = model_count.predict()
                probi = model_zero.predict()
                probi = probi/(probi + (1-probi)*st.nbinom.pmf(0,*convert_params(theta = 1, mu = mui)))
                probi[Y1] = 0                

                ll_new = loglikfun(np.hstack((start['count'].values,start['zero'].values)))
                
        if (EM is True) and (dist is 'negbin'):
            warnings.warn('EM estimation of starting values not available for Negetive Binomial.')
            #mui = model_count.predict() # or model_count.mu
            #probi = model_zero.predict()
            #probi = probi/(probi + (1-probi)*st.nbinom.pmf(0,*convert_params(theta = start['theta'], mu = mui)))
            #probi[Y1] = 0
            
            #ll_new = loglikfun(np.hstack((start['count'].values,start['zero'].values,np.log(start['theta']))))
            #ll_old = 2 * ll_new 
            
            #while np.absolute((ll_old - ll_new)/ll_old) > reltol :
            #    ll_old = ll_new
            #    model_count = sm.GLM(endog = Y, exog = X, family = \
            #                         sm.families.NegativeBinomial(alpha = start['theta']),\
            #                      offset = offsetx , freq_weights = weights*(1-probi), \
            #                          start_params = start['count']).fit()
            #    model_zero = sm.GLM(probi, exog = Z, family=sm.families.Binomial(link = linkobj.linkclass),\
            #            offset = offsetz, freq_weights = weights, \
            #            start_params = start['zero']).fit()
            #    start = {'zero':model_zero.params, 'count':model_count.params, 'theta':model_count.scale}
    
            #    mui = model_count.predict()
            #    probi = model_zero.predict()
            #    probi = probi/(probi + (1-probi)*st.nbinom.pmf(0,*convert_params(theta = start['theta'], mu = mui)))
            #    probi[Y1] = 0

            #    ll_new = loglikfun(np.hstack((start['count'].values,start['zero'].values,np.log(start['theta']))))
    
    ## ML Estimation
    if (dist is 'negbin'):
        x0 = np.hstack((start['count'].values,start['zero'].values,\
                                         np.log(start['theta'])))
    else:
        x0 = np.hstack((start['count'].values,start['zero'].values))

    fit = sp.optimize.minimize(loglikfun, args=(-1.0,), x0 = x0, method=method, jac=gradfun, options=options)
    
    ## coefficients and covariances
    coefc = pd.Series(data = fit.x[0:kx], index = X.columns.values)
    coefz = pd.Series(data = fit.x[kx:kx+kz], index = Z.columns.values)

    if method == 'L-BFGS-B':
        vc_data = -fit.hess_inv.todense()
    elif method == 'BFGS':
        vc_data = -fit.hess_inv
    else:
        warnings.warn('Not tested for methods other than BFGS and L-BFGS-B.')
    vc = pd.DataFrame(data = vc_data[np.arange(kx+kz)[:,None],np.arange(kx+kz)], \
                      index = np.append(X.columns.values, Z.columns.values),\
                 columns = np.append(X.columns.values, Z.columns.values))
    if dist == 'negbin':
        ntheta = kx + kz
        theta = np.exp(fit.x[ntheta])
        # Can hessian term for theta be negative? Using absolute here.
        SE_logtheta = np.sqrt(np.abs(np.diagonal(vc_data)[ntheta]))
    else:
        theta = None
        SE_logtheta = None
    
    ## fitted and residuals
    mu = np.exp(np.dot(X,coefc)+offsetx)
    phi = linkobj.link_inv(np.dot(Z,coefz)+offsetz)
    Yhat = (1-phi) * mu
    res = np.sqrt(weights) * (Y - Yhat)

    ## effective observations
    nobs = np.sum(weights > 0)
    
    start= {'count': coefc, 'zero' : coefz, 'theta': theta}
    
    
    
        
    
    return start

In [ ]:
#class zipModel(object):
    
     #   def __init__(self):
        
            self.converged = fit.success
            self.iters = fit.nit;                   # number of iterations for convergence
            self.loglike = fit.fun;                 # log-likelihood
        
            self.coeff_count = coefc  # coefficient vector for count model
            self.coeff_zero = coefz   # coefficient vector for zero-inflation model
        
            self.theta = theta if (dist is 'negbin') else None
        
            self.residuals = np.array(res)    # function needs to be implemented
            self.vcov = vc
        
            # string vars
            self.var_names_count = X.columns.values       # variable names, intercept in feat_names[0]
            self.var_names_zero = Z.columns.values        # var name for zero-inflated variable
            self.response_name = y.columns.values[0]          # name of the response variable
        
            self.call = ''
            
            self.coefficients = {'count':coefc ,'zero': coefz}
            self.residuals = res
            self.fitted_values = Yhat
            self.optim = fit
            self.method = method
            self.comtrol = ''
            self.start = start
            self.weights = []
            self.offset = []
            self.n = nobs
            self.df_null = nobs - 2
            self.df_residual = 
            self.terms = ''
            self.theta = theta
            self.SE_logtheta = 
            self.loglik = ''
            self.vcov = vc
            self.dist = dist
            self.link = linkstr
            self.linkinv = 
            self.converged = fit.success
            self.call = ''
            self.formula = ''

In [3]:
### Lines below will not go inside the function
import numpy as np
import pandas as pd
import patsy
df = pd.read_csv('DebTrivedi.csv',index_col = [0])
sel = np.array([1, 6, 7, 8, 13, 15, 18])-1
df = df.iloc[:,sel]
# produce design matrices from R-style formula
X_formula = 'ofp ~ hosp + health + numchron + gender + school + privins'
y, X = patsy.dmatrices(X_formula, df, return_type='dataframe')
Z_formula = 'ofp ~ health'
Z = patsy.dmatrices(Z_formula, df, return_type='dataframe')[1]
zeroinfl(y,X,Z,dist='negbin',method='BFGS')

/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:362: UserWarning: EM estimation of starting values not available for Negetive Binomial.


{'count': Intercept              0.941806
 health[T.excellent]   -0.329442
 health[T.poor]         0.328911
 gender[T.male]        -0.124682
 privins[T.yes]         0.218386
 hosp                   0.222516
 numchron               0.173664
 school                 0.026673
 dtype: float64,
 'theta': 1.2587589747287193,
 'zero': Intercept             -5.043172
 health[T.excellent]    1.082341
 health[T.poor]         1.619519
 dtype: float64}

In [ ]:
coefficients = list(count = coefc, zero = coefz),
    residuals = res,
    fitted.values = Yhat,
    optim = fit,
    method = method,
    control = ocontrol,
    start = start,
    weights = if(identical(as.vector(weights), rep.int(1L, n))) NULL else weights,
    offset = list(count = if(identical(offsetx, rep.int(0, n))) NULL else offsetx,
      zero = if(identical(offsetz, rep.int(0, n))) NULL else offsetz),
    n = nobs,
    df.null = nobs - 2,
    df.residual = nobs - (kx + kz + (dist == "negbin")),
    terms = list(count = mtX, zero = mtZ, full = mt),
    theta = theta,
    SE.logtheta = SE.logtheta,
    loglik = fit$value,
    vcov = vc,
    dist = dist,
    link = linkstr,
    linkinv = linkinv,
    converged = fit$convergence < 1,
    call = cl,
    formula = ff,